In [1]:
from __future__ import print_function


import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from skimage.io import imshow
from keras import backend as K
from keras.constraints import Constraint
from keras.datasets import fashion_mnist
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):

    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == np.argmax(label_vec1[num]):
            acc1 += 1

    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == np.argmax(label_vec2[num]):
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)

    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )


    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('maximum inference accuracy is:', max_accuracy)
    print('precision is:', best_precision)
    print('recall is:', best_recall)
    return max_accuracy


In [0]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}


def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/dataset_purchase" 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
print("Loading dataset...")
data_set =np.genfromtxt(path,delimiter=',')
print("Dataset loaded.")

Loading dataset...
Dataset loaded.


In [0]:
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, precision_score, recall_score

X = data_set[:15000,1:].astype(np.float64)
Y = (data_set[:15000,0]).astype(np.int32)-1

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [7]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(600,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(100, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              615424    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               12900     
Total params: 1,317,348
Trainable params: 1,317,348
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on

In [8]:
output_train=model.predict(x_train)
output_test=model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (1.0, 0.7628888888888888)
maximum inference accuracy is: 0.7251587301587301
precision is: 0.573376509171417
recall is: 1.7594285714285713
Maximum Accuracy: 0.7251587301587301


# **Sparsity**

In [0]:
masks = create_sparsity_masks(model,20)#Closest 30% to 0

sparse_model = Sequential()
sparse_model.add(Dense(1024,kernel_constraint=Sparse(masks[0]), activation='relu', input_shape=(600,)))
sparse_model.add(Dense(512,kernel_constraint=Sparse(masks[1]), activation='relu'))
sparse_model.add(Dense(256,kernel_constraint=Sparse(masks[2]), activation='relu'))
sparse_model.add(Dense(128,kernel_constraint=Sparse(masks[3]), activation='relu'))
sparse_model.add(Dense(100,kernel_constraint=Sparse(masks[4]), activation='softmax'))

adam = Adam()
sparse_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
sparse_model.summary()
#Get weights from densely trained model
sparse_model.set_weights(model.get_weights())


# Train Sparse Model
sparse_model.fit(x_train,y_train,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test))


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1024)              615424    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               12900     
Total params: 1,317,348
Trainable params: 1,317,348
Non-trainable params: 0
_________________________________________________________________
Train on 10500 samples, validate on 4500 samples
Epoch 1/50
10500/10500 [==============================] - 1s 

In [0]:
output_train=sparse_model.predict(x_train)
output_test=sparse_model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)